# Build a Simple LLM Application

In this quickstart, we'll show how to build a simple LLM application.

This application will translate text from English into another language.

This is a relatively simple LLM application: it's just a single LLM call plus some prompting.

Still, this is a great way to get started with LangChain-a lot of features can be built with just some prompting and an LLM call!

# Concepts

Concepts we will cover are:
- Using [**language models**](https://python.langchain.com/v0.2/docs/concepts/#chat-models)
- Using [**PromptTemplates**](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) and [**OutputParsers**](https://python.langchain.com/v0.2/docs/concepts/#output-parsers)
- [**Chaining**](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language) a PromptTemplate + LLM + OutputParser using LangChain
- Debugging and tracing your application using [**LangSmith**](https://python.langchain.com/v0.2/docs/concepts/#langsmith)
- Deploying your application with [**LangServe**](https://python.langchain.com/v0.2/docs/concepts/#langserve)

# Imports and Setup

## Loading Credentials

In [5]:
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

## LangSmith

In [7]:
!export LANGCHAIN_TRACING_V2="true"

# Using Language Models

In [8]:
from langchain_openai import ChatOpenAI

In [9]:
model = ChatOpenAI(model="gpt-4")

Let's first use the model directly.

`ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them.

To just simply call a model, we can pass in a list of messages to the `invoke` method.

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

In [11]:
messages = [
    SystemMessage(content="Translate the following from English into Italian."),
    HumanMessage(content="Hi!")
]
model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49a7f10d-eb5e-4cfe-8f5f-661472698e7e-0', usage_metadata={'input_tokens': 21, 'output_tokens': 3, 'total_tokens': 24})

If LangSmith is enabled, we can see that is run is logged to LangSmith, and can see the [**LangSmith trace**](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)

# Output Parsers

Notice that the response from the model is an `AIMessage`.

This contains a string response along with other metadata about the response.

Often, we may just want to work only with the string response.

**We can parse out just this response by using a simple output parser**.

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
parser

StrOutputParser()

In [16]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

More commonly, we can ***"chain"*** the model with this output parser.

This means this output parser will get called everythime in this chain.

This chain...
- takes on the input type of the language model (string or list of messages)
- returns the output type of the output parser (here: string)

We can easily create the chain using the `|` operator. It is used in LangChain to combine several elements together.

In [17]:
chain = model | parser

In [18]:
chain

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x733ccff9c4c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x733ccff9dbd0>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [19]:
chain.invoke(messages)

'Ciao!'

If we now look at LangSmith, we can see that the chain has two steps:
- first, the language model is called,
- then, the result is passed to the output parser.

We can see the [**LangSmith trace**](https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r)

# Prompt Templates

Right now, we are passing a list of messages directly into the language model.

Where does this list of messages come from?

Usually, it's constructed from a combination of user input and application logic. This application logic usually takes the ra user input and transforms it into a list of meassages ready to pass to the language model.

Common transformations include:
- adding a system message,
- formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation.

They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It will take in two user variables:
- `language`: the language to translate text into,
- `text`: the text to translate.

In [20]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that we will format to be the system message.

In [21]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `system_template` as well as a simpler template for where to put the text.

In [22]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

The input to this prompt template is a **dictionary**.

We can play around with this prompt template by itself to see what it does by itself.

In [23]:
result = prompt_template.invoke({
    "language": "italian",
    "text": "hi"
})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

We can see that it returns a `ChatPromptValue` that consists of two messages.

If we want to access the message directly, we do:

In [24]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

We can now combine this with the model, and the output parser from above.

This will chain all the three component together.

In [25]:
chain = prompt_template | model | parser

In [26]:
chain

ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate the following into {language}:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x733ccff9c4c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x733ccff9dbd0>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [27]:
chain.invoke({
    "language": "italian",
    "text": "hi"
})

'ciao'

If we take a look at the LangSmith trace, we can see all the three components show up in the [**LangSmith trace**](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r)

# Serving with LangServe

Now that we've built an application, we need to serve it. That's where **LangServe** comes in.

LangServe helps developers deploy LangChain chains as REST API. You do not need to use LangServe to use LangChain, but, in this guide, we'll show you how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter notebook or script, we will new move our of that. We will be creating a Python file and then interacting with it from the command line.

## Server

To create a server for our application, we'll make a `serve.py` file.

This will contain our logic for serving our application.

It consists of three things:
1. The definition of our chain that we just built above.
2. Our FastAPI app.
3. A definition of a route from which to serve the chain, which is done with `langserve.add_routes`

You then have to run this file. And the chain will be served at `http://localhost:8000`

# Playground

Every LangServe service comes with a simple [**built-in UI**](https://github.com/langchain-ai/langserve/blob/main/README.md#playground) for configuring and invoking the application with streaming output and visibility into intermediate steps.

Head to `http://localhost:8000/chain/playground/` to try it out!

# Client

Now, let's set up a client for programmatically interacting with our service.

We can easily do this with the `langserve.RemoteRunnable`